In [1]:
import ee


In [2]:

##ee.Authenticate()
ee.Initialize()
import geemap
Map = geemap.Map(center=[36,-120], zoom =8)


Make stack of LANDFIRE inputs:
    BPS code
    BPS name
    BPS model
    EVH
    EVC
    Lat/long

In [43]:
#AOI
aoi = ee.Geometry.BBox(-119.62601, 36.98082 , -119.12596,37.23979)
#Start geemap 
Map = geemap.Map(center=[36,-120], zoom =8)


bps = ee.ImageCollection("LANDFIRE/Vegetation/BPS/v1_4_0").reduce(ee.Reducer.max())
#rapcovers = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
evh = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
rapcoverha = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('AFG')
rapcoverhp = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('PFG') 
rapcovers = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('SHR') 
sclass = ee.ImageCollection("LANDFIRE/Fire/SClass/v1_4_0").reduce(ee.Reducer.max())

rapcoverh = rapcoverha.add(rapcoverhp)

col = ee.ImageCollection([])
for year in range(2019,2023):
    # Define the GeoTIFF file path in the GCS bucket
    geotiff_path = f'gs://gigafire_rvs/rpms/rpms_{year}.tif' 
    
    # Load the GeoTIFF as an image
    image = ee.Image.loadGeoTIFF(geotiff_path)
    
    # Add the image to the image collection
    col = col.merge(ee.ImageCollection([image]))

# Print the image collection
print("Image Collection:", col.getInfo())


percentiles = [5, 25, 50, 75, 95]


rpms_percentiles = col.reduce(ee.Reducer.percentile(percentiles))

col = ee.ImageCollection("LANDSAT/LC08/C01/T1_32DAY_NDVI")

def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_ndvi = col.filterDate(start_date, end_date).median().set('year', year)
    return annual_ndvi
    
    # Create annual composites for each year in the range
start_year = 2013
end_year = 2021
annual_ndvi_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

# Compute percentiles
ndvi_percentiles = annual_ndvi_collection.reduce(ee.Reducer.percentile(percentiles))

#Gridmet precip
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select(['pr'])

#Gridmet annual sums
def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_precip = gridmet.filterDate(start_date, end_date).sum().set('year', year)
    return annual_precip
    
    # Create annual composites for each year in the range
start_year = 1979
end_year = 2023
annual_precip_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

precip_percentiles = annual_precip_collection.reduce(ee.Reducer.percentile(percentiles))


landfire_stack = ee.Image(
        [bps, rapcoverh, evh, ndvi_percentiles, precip_percentiles, sclass,rpms_percentiles,rapcovers]).toFloat()








Image Collection: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [136595, 87376], 'crs': 'GEOGCS["WGS 84", \n  DATUM["World Geodetic System 1984", \n    SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n    AUTHORITY["EPSG","6326"]], \n  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Geodetic longitude", EAST], \n  AXIS["Geodetic latitude", NORTH], \n  AUTHORITY["EPSG","4326"]]', 'crs_transform': [0.00026949458523585647, 0, -124.84902599680046, 0, -0.00026949458523585647, 49.38461324988546]}], 'properties': {'system:index': '1_1_1_2_0'}}, {'type': 'Image', 'bands': [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [136595, 87376], 'crs': 'GEOGCS["WGS 84", \n  DATUM["World Geodetic System 1984", \n   

In [44]:
from google.cloud import storage

bucket_id = "gigafire_rvs"

client = storage.Client()
bucket = client.get_bucket(bucket_id)
# list all objects in the directory
blobs = bucket.list_blobs(prefix="lf")
for blob in blobs:
    blob.delete()

#Export tiles to bucket (shard size 256)
def export_stack_to_cloud_storage(stack):
    # ee_region_bbox = ee_region.bounds()
    # print(ee_region_bbox.coordinates())
    task = ee.batch.Export.image.toCloudStorage(
        image=stack,
        fileNamePrefix="lf",
        bucket=bucket_id,
        scale=30,
        crs='EPSG:4326',
        shardSize=256,
        region=aoi,
        fileDimensions=256,
        skipEmptyTiles=True,
        maxPixels=1e13,
        fileFormat="GeoTIFF",
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()

export_stack_to_cloud_storage(landfire_stack)


In [3]:
###Download from bucket to tmp folder

from google.cloud import storage
import os
import tempfile

bucket_id = "gigafire_rvs"
#wd = "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/temp"
# Create a temporary directory to download the files
temp_dir = tempfile.mkdtemp()
print(temp_dir)

def list_files_in_bucket(bucket_name):
    """List all files in a Google Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    files = []
    blobs = bucket.list_blobs(prefix="lf")
    for blob in blobs:
        files.append(blob.name)

    return files

def download_files_from_bucket(bucket_name, local_temp_dir):
    """Download files from a Google Cloud Storage bucket to a local directory."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    local_files = []
    blobs = bucket.list_blobs(prefix="lf")
    for blob in blobs:
        local_file_path = os.path.join(local_temp_dir, blob.name)
        blob.download_to_filename(local_file_path)
        local_files.append(local_file_path)

    return local_files




# Download files to temporary directory
local_files = download_files_from_bucket(bucket_id, temp_dir)
print("Downloaded files:", local_files)





C:\Users\ingli\AppData\Local\Temp\tmpjwccs6i1
Downloaded files: ['C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000000512.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000000768.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000001024.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000001280.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000001536.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000000-0000001792.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000256-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000256-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmpjwccs6i1\\lf0000000256-0000000512.tif', 'C:\\Users\

In [4]:
##Geotiff to dataframe

import rasterio
import pandas as pd




def multiband_geotiff_to_dataframe(geotiff_path):
    """Converts a multiband GeoTIFF file into a pandas DataFrame with x and y coordinates."""
    # Open the GeoTIFF file
    with rasterio.open(geotiff_path) as src:
        # Read all bands
        num_bands = src.count
        band_data = [src.read(i) for i in range(1, num_bands + 1)]
        
        # Get metadata
        transform = src.transform
        height, width = band_data[0].shape

        # Generate x and y coordinates
        x_coords = []
        y_coords = []
        for i in range(height):
            for j in range(width):
                x, y = src.xy(i, j)
                x_coords.append(x)
                y_coords.append(y)

        # Flatten band data and coordinates into DataFrame
        df_data = {'x': x_coords, 'y': y_coords}
        for band_idx in range(num_bands):
            band_name = src.descriptions[band_idx] if src.descriptions else f'band_{band_idx + 1}'
            df_data[band_name] = band_data[band_idx].flatten()

        df = pd.DataFrame(df_data)

    return df


dfs_1 = [multiband_geotiff_to_dataframe(file_path) for file_path in local_files]

for df in dfs_1:
    # Identify NDVI columns
    ndvi_columns = [col for col in df.columns if col.startswith('NDVI_')]
    
    # Apply the power operation to NDVI columns
    if ndvi_columns:  # Check if there are any NDVI columns
        df[ndvi_columns] = df[ndvi_columns].apply(lambda x: x * 10000)

print(dfs_1[1].head())

# Clean up temporary directory
## for file_path in local_files:
    ##vos.remove(file_path)
## os.rmdir(temp_dir)




            x          y  BPS_max   AFG  EVH_max      NDVI_p5     NDVI_p25  \
0 -119.557096  37.239705    587.0  10.0    109.0  2506.012207  3291.484619   
1 -119.556826  37.239705    587.0  32.0    109.0  2560.038574  3495.035889   
2 -119.556557  37.239705    588.0  48.0    106.0  2329.834229  2882.511719   
3 -119.556287  37.239705    587.0  56.0    109.0  2230.282959  2981.898193   
4 -119.556018  37.239705    587.0  59.0    110.0  2430.759033  3159.838623   

      NDVI_p50     NDVI_p75     NDVI_p95  ...      pr_p50      pr_p75  \
0  3996.831787  4104.953613  4647.799805  ...  789.761475  1009.69165   
1  3847.980469  4103.824219  4922.839844  ...  789.761475  1009.69165   
2  3255.186768  3828.988037  4274.503418  ...  789.761475  1009.69165   
3  3409.436279  3750.000000  3884.132568  ...  789.761475  1009.69165   
4  3720.214844  4203.349121  4479.464355  ...  789.761475  1009.69165   

        pr_p95  SClass_max  B0_p5  B0_p25  B0_p50  B0_p75  B0_p95   SHR  
0  1390.079224    

In [5]:
import pandas as pd
##Join df to kattribute table and reorder and clean columns
key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140BPS_01122015.csv")
height_key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140EVH_05092014.csv")

def join_new_attributes(df):  
    join = df.join(key[['VALUE','BPS_CODE', 'BPS_NAME','BPS_MODEL']].set_index('VALUE'), on='BPS_max')
    join = join.join(height_key[['VALUE','height']].set_index('VALUE'), on='EVH_max', how="outer")
    join = join.assign(PLOT_ID=range(len(join)))
    cols = list(join.columns)
    Plots_Active = join[['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 
                     'AFG', 'height',
                     'y', 'x', *cols[10:15], *cols[5:10], 'SClass_max', *cols[16:21]]]
    Plots_Active['BPS_CODE'] = Plots_Active['BPS_CODE'].fillna(0).astype(int)
    return Plots_Active


dfs = [join_new_attributes(df) for df in dfs_1]
print(["joined attributes to", len(dfs), "dfs"])


#df['Column1'] = df['Column1'].astype(int)


#for i, df in enumerate(dfs):
    #df.to_csv(f'rvs_inputdf_{i}.csv', index=False)  

['joined attributes to', 32, 'dfs']


In [6]:
import numpy as np

#Join bps code to shrub cover to shrub info from combined growth rates table
key_shrubs = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/rvs/BPS_Combined_Growthrates.csv")
key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140BPS_01122015.csv")
height_key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140EVH_05092014.csv")
key['BPS_MODEL'] = key['BPS_MODEL'].replace(['na', 'nan'], np.nan)
key['BPS_MODEL'] = key['BPS_MODEL'].astype(float)
species_key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/Bio_Crosswalk.csv")
filtered_shrubs = key_shrubs[
    (key_shrubs['COMBINED_TYPE'].str.contains('S')) & 
    (key_shrubs['COVER_TYPE'] == 'Late') &
    (key_shrubs['ZONE'] == 6)]

#species_key.set_index('spp_code', inplace=True)
#species_key = species_key[~species_key.duplicated(keep='first')]

def make_shrubs_table(df):  
    join = df.join(key[['VALUE','BPS_CODE', 'BPS_NAME','BPS_MODEL']].set_index('VALUE'), on='BPS_max')
    join = join.join(height_key[['VALUE','height']].set_index('VALUE'), on='EVH_max', how="outer")
    join = join.assign(PLOT_ID=range(len(join)))
    join = pd.merge(
        join,
        filtered_shrubs[['BPS_CODE', 'BPS_MODEL', 'Species_1','Species_2','Species_3','Species_4']],
     on=['BPS_MODEL', 'BPS_CODE'],
        how='left'
    )
    Shrubs_Active = join[['PLOT_ID', 'Species_1', 'Species_2','Species_3','Species_4','SHR', 'height', 'BPS_CODE', 'BPS_NAME']]
    Shrubs_Active['BPS_CODE'] = Shrubs_Active['BPS_CODE'].fillna(0).astype(int)
    # Insert new second column with all zeros (as string)
    Shrubs_Active.insert(1, 'Plot_Name', '0')
    lifeform_dict = species_key.set_index('spp_code')['lifeform'].to_dict()
    for col in ['Species_1', 'Species_2', 'Species_3', 'Species_4']:
        Shrubs_Active[col + '_Lifeform'] = Shrubs_Active[col].map(lifeform_dict)
    
    def find_first_shrub_species(row):
        if row['Species_1_Lifeform'] == 'Shrub':
            return row['Species_1']
        elif row['Species_2_Lifeform'] == 'Shrub':
            return row['Species_2']
        elif row['Species_3_Lifeform'] == 'Shrub':
            return row['Species_3']
        elif row['Species_4_Lifeform'] == 'Shrub':
            return row['Species_4']
        else:
            return pd.NA # Return NA if no shrub species is found

    Shrubs_Active['dom_spp'] = Shrubs_Active.apply(find_first_shrub_species, axis=1)
    Shrubs_Active['spp_code'] = Shrubs_Active['dom_spp']
    Shrubs_Active = Shrubs_Active[['PLOT_ID', 'Plot_Name', 'dom_spp', 'spp_code', 'SHR', 'height', 'BPS_CODE','BPS_NAME']]
    Shrubs_Active.rename(columns={'SHR': 'cover'}, inplace=True)
    Shrubs_Active = Shrubs_Active.dropna(subset=['spp_code'])
    
    return Shrubs_Active

 


dfs_shrubs= [make_shrubs_table(df) for df in dfs_1]

print(dfs_shrubs[1])

       PLOT_ID Plot_Name dom_spp spp_code  cover  height  BPS_CODE  \
53          53         0    ADFA     ADFA    7.0     NaN     11050   
54          54         0    ADFA     ADFA   33.0     NaN     11050   
57          57         0    ADFA     ADFA    8.0     NaN     11050   
58          58         0    ADFA     ADFA    8.0     NaN     11050   
64          64         0    ADFA     ADFA    7.0     NaN     11050   
...        ...       ...     ...      ...    ...     ...       ...   
65451    65451         0    ADFA     ADFA   30.0     NaN     11050   
65452    65452         0    ADFA     ADFA   18.0     NaN     11050   
65510    65510         0    ADFA     ADFA   16.0     NaN     11050   
65511    65511         0    ADFA     ADFA   22.0     NaN     11050   
65516    65516         0    ADFA     ADFA   27.0     NaN     11050   

                                                BPS_NAME  
53     Northern and Central California Dry-Mesic Chap...  
54     Northern and Central California Dr

In [14]:
import sqlite3
import pandas as pd
import os
import shutil
import time

# Copy template to home folder for reading and writing 
source_file = 'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/test/rvs_demo_fromRobbAddFieldTypeMissing.db'
counter=0
n = dfs[0].shape[0]

for i in dfs:
    counter=counter+1
    destination_file = f'C:/Users/ingli/Desktop/home/240625_shrubs_{counter}.db'

    if os.path.exists(destination_file):
        os.remove(destination_file)

    shutil.copy(source_file, destination_file)
    print("Template DB copied successfully.# Path to the SQLite database file")
    database_path = destination_file
    print(database_path)

# Create a connection to the database
    conn = sqlite3.connect(database_path)

# Define the table name
    table_name = "Plots_Active"

# Get the column names of the table
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    cols= [column[1] for column in cursor.fetchall()]  + ['NPP_1', 'NPP_2', 'NPP_3', 'NPP_4', 'NPP_5']
    print(cols)
    # Read data from pandas DataFrame
    # Assuming df is your pandas DataFrame with the same column names


    i.columns=cols
    # Replace values in the table with pandas DataFrame values
    i.to_sql(table_name, conn, if_exists='replace', index=False)


    cursor.execute(f"PRAGMA table_info({table_name})")
    columns_info = cursor.fetchall()
    npp_columns = [column[1] for column in columns_info if column[1].startswith("NPP")]

    # Execute UPDATE statements to set NULL values to 0 for each NPP column
    for column_name in npp_columns:
        update_query = f"UPDATE {table_name} SET {column_name} = 0 WHERE {column_name} IS NULL"
        cursor.execute(update_query)


    table_name="Disturbance_Plots_NoFire"
    cursor.execute(f"DROP TABLE {table_name}")

    table_name="Dist_year7fire"
    cursor.execute(f"DROP TABLE {table_name}")

    table_name="Disturbance_Plots"
    cursor.execute(f"DROP TABLE {table_name}")
    # Get the list of column names that start with "NPP"
 


    # Create a cursor object using the cursor() method
    #cursor.execute("DELETE FROM Disturbance_Plots")
    # Insert rows with PLOT_IDs from 1 to N
   ## for plot_id in range(1, n + 1):
       ## insert_data_query = f'''
       ## INSERT OR REPLACE INTO Disturbance_Plots (PLOT_ID, DIST_TYPE, DIST_SUBTYPE, P1_VAL, FREQ, START_YEAR, STOP_YEAR)
       ## VALUES ({plot_id}, 'FIRE', 'WILDFIRE', 100.0, 0, 5, 5)
       ## '''
       ## cursor.execute(insert_data_query)

    #conn.execute(f"UPDATE Plots_Active SET sclass = 10") 
    
    conn.commit()
    conn.close()
    # Commit changes and close connection

counter=0
n = dfs_shrubs[0].shape[1]
print(n)
for i in dfs_shrubs:
    counter=counter+1
    database_path = f'C:/Users/ingli/Desktop/home/240625_shrubs_{counter}.db'


# Create a connection to the database
    conn = sqlite3.connect(database_path)

    table_name = "Shrubs_Active"

# Get the column names of the table
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    cols= [column[1] for column in cursor.fetchall()] 
    print(cols)
    # Read data from pandas DataFrame
    # Assuming df is your pandas DataFrame with the same column names
    i.columns=cols[0:n]
    # Replace values in the table with pandas DataFrame values
    i.to_sql(table_name, conn, if_exists='replace', index=False) 
    print("Shrubs table added")
    conn.commit()
    conn.close()
#Copy file from home to RVS git folder for RVS use
    source_file = f'C:/Users/ingli/Desktop/home/240625_shrubs_{counter}.db'
    destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/inputs/240625_shrubs_{counter}.db'
    shutil.copy(source_file, destination_file)
    print("Final DB copied successfully to inputs")


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/ingli/Desktop/home/240625_shrubs_1.db'

In [12]:
conn.commit()
conn.close()

In [10]:
import pandas as pd
import os
import shutil
import time
import sqlite3

rows = [500]

# DataFrame to store execution times


# Number of iterations

# DataFrame to store execution times
execution_times_df = pd.DataFrame(columns=['Rows', 'Execution Time (seconds)'])

for i in rows:
    
    destination_file = f'C:/Users/ingli/Desktop/home/240625_shrubs_1.db'


    conn = sqlite3.connect(destination_file)
    cursor = conn.cursor()
    

    cursor.execute(f'''
    DELETE FROM Plots_Active
    WHERE PLOT_ID NOT IN (
        SELECT PLOT_ID FROM Plots_Active
        ORDER BY PLOT_ID ASC
        LIMIT {i}
    )
    ''')

    cursor.execute(f'''
    DELETE FROM Shrubs_Active
    WHERE PLOT_ID NOT IN (
        SELECT PLOT_ID FROM Shrubs_Active
        ORDER BY PLOT_ID ASC
        LIMIT {i}
    )
    ''')
        
    conn.commit()
    conn.execute('VACUUM')
    # Close the database connection
    conn.close()


    source_file = f'C:/Users/ingli/Desktop/home/240625_shrubs_1.db'
    destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/240625_shrubs_500.db'
    shutil.copy(source_file, destination_file)
    print("File copied successfully.")



OperationalError: database is locked

In [ ]:
##ADD fire at YEAR 5 


database_path = f'C:/Users/ingli/Desktop/home/240529_landfire_plots_cover_1.db'

# Create a connection to the database
conn = sqlite3.connect(database_path)

# Define the table name
    
n = dfs[0].shape[0]
print(n)

# Create a cursor object using the cursor() method
cursor = conn.cursor()
cursor.execute("DELETE FROM Disturbance_Plots")
# Insert rows with PLOT_IDs from 1 to N
for plot_id in range(0, n + 1):
    insert_data_query = f'''
INSERT OR REPLACE INTO Disturbance_Plots (PLOT_ID, PLOT_NAME, DIST_TYPE, DIST_SUBTYPE, P1_VAL, FREQ, START_YEAR, STOP_YEAR,P2_VAL, P3_VAL)
VALUES ({plot_id}, '{plot_id}', 'FIRE', 'WILDFIRE', 100.0, 0, 5, 5,0,0)
 '''
    cursor.execute(insert_data_query)
    print(plot_id)
conn.commit()
# Close the connection
conn.close()


destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/inputs/240529_landfire_plots_cover_1_fire5.db'
shutil.copy(database_path, destination_file)
print("Final fire  DB copied successfully to inputs")

In [42]:


!docker run -v "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test":"/data" rvs-image-2024-dev /bin/bash -c "rvs /data/240625_shrubs_500.db /data/out.db 10 true; cp RVS_Debug.txt /data/"

rvs
/data/240625_shrubs_500.db
/data/out.db
10
true
3.45.1
2024-01-30 16:01:20 e876e51a0ed5c5b3126f52e532044363a014bc594cfefa87ffb5b82257ccalt1
Loading records...
Loading disturbances...
Done.

YEAR 0

Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Biomass reduction total (driver to Biomass): 0
Disturbed?:  0
Disturb about from BiomassDIO: 0
Bioma